In [ ]:
#### Coutresy to convert XML to JSON https://stackoverflow.com/questions/191536/converting-xml-to-json-using-python 
Made small modifications for our purpose
Remove certain first few lines from XML files and run the code by changing the XML filename

In [ ]:
from xml.dom import minidom
import simplejson as json
def parse_element(element):
    dict_data = dict()
    if element.nodeType == element.TEXT_NODE:
        if element.data != " ":
            dict_data['data'] = element.data
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_NODE, 
                                element.DOCUMENT_TYPE_NODE]:
        for item in element.attributes.items():
            dict_data[item[0]] = item[1]
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_TYPE_NODE]:
        for child in element.childNodes:
            child_name, child_dict = parse_element(child)
            if child_name in dict_data:
                try:
                    dict_data[child_name].append(child_dict)
                except AttributeError:
                    dict_data[child_name] = [dict_data[child_name], child_dict]
            else:
                dict_data[child_name] = child_dict 
    return element.nodeName, dict_data

if __name__ == '__main__':
    dom = minidom.parse('Q2Q.xml')
    print type(dom)
    f = open('Q2Q.json', 'w')
    f.write(json.dumps(parse_element(dom), sort_keys=True, indent=4))
    f.close()

In [ ]:
import pandas as pd

data111 = pd.read_json('Q2Q.json')
print data111[0][1]['xml'].keys()
df3111 = pd.io.json.json_normalize(data111[0][1]['xml']['OrgQuestion'])
df4111 = pd.concat([pd.DataFrame(pd.io.json.json_normalize(x)) for x in df3111['Thread.RelComment']],ignore_index=True)
df4111['THREAD_SEQUENCE'] = [ '_'.join(i.split('_')[:2]) for i in df4111['RELC_ID']]
print df4111.count()

In [ ]:
df5111 = pd.concat([df3111.set_index('Thread.RelQuestion.RELQ_ID'),df4111.set_index('THREAD_SEQUENCE')], axis=1, join='inner').reset_index()
df5111.columns

remove_index = [ '#text', 'OrgQBody.#text.data' ,'Thread.#text', 'Thread.RelQuestion.#text', 'Thread.RelQuestion.RelQBody.#text.data' 
               ,'Thread.SubtaskA_Skip_Because_Same_As_RelQuestion_ID' , 'index' , 'RelCBody.#text.data' , 'Thread.RelComment']

[ df5111.drop(i, axis=1, inplace=True) for i in remove_index ] 
list ( df5111[ df5111['ORGQ_ID'] == 'Q268' ].columns ) 